<a href="https://colab.research.google.com/github/olodenm/info_2022_olo/blob/Netology_HW/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BA_%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D0%B8_%C2%AB%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_%D0%B2%D0%B5%D0%B1_%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0%C2%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обязательная часть
Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/). Функция в качестве параметра должна принимать **список** запросов для поиска (например, ```['python', 'анализ данных']```) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from urllib.parse import quote
import urllib.parse

In [2]:
# метод get
res = requests.get('https://habr.com/ru')
res
# res.status_code

<Response [200]>

In [3]:
# браузер отрисовал бы страницу на основе данного текста
res.text

'<!DOCTYPE html>\n<html lang="ru" data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D">\n<head >\n  <meta charset="UTF-8">\n  <meta name="viewport" content="width=device-width,initial-scale=1.0,viewport-fit=cover,maximum-scale=1,user-scalable=0">\n  <meta name="referrer" content="unsafe-url">\n  <title>Все статьи подряд / Хабр</title>\n  <style>\n    /* cyrillic-ext */\n    @font-face {\n      font-family: \'Fira Sans\';\n      font-style: normal;\n      font-weight: 500;\n      font-display: swap;\n      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format(\'woff2\');\n      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;\n    }\n\n    /* cyrillic */\n    @font-face {\n      font-family: \'Fira Sans\';\n      font-style: normal;\n      font-weight: 500;\n      font-display: swap;\n      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format(\'woff2\');\n    

In [ ]:
# создаем объекта, через методы которого будем искать нужные теги и извлекать их содержимое
soup = BeautifulSoup(res.text, 'html.parser')
soup

In [ ]:
# функция find_all позволяет найти все указанные тег с нужными атрибутами (с вложениями), возвращает список
news = soup.find_all('div', class_='tm-articles-list')
print(len(news))
print(news[0])

In [6]:
def habr(query):
    df = pd.DataFrame() # Создаем пустой DataFrame для хранения результатов
    URL = 'https://habr.com/ru/search/' # URL сайта Habr, на котором будем выполнять поиск
    for q in query:
        params = {
            'q': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:  # Проходим по каждой найденной статье
            title = article.find('h2', class_="tm-title tm-title_h2").text # заголовок
            link = article.find('a', class_="tm-title__link").get('href') # ссылка
            date = article.find('span', class_="tm-article-datetime-published").text # дата
            row = {'date': date, 'title': title, 'link': link}
            df = pd.concat([df, pd.DataFrame([row])])

    return df.reset_index(drop=True) # Сбрасываем индекс DataFrame и возвращаем результат

result = habr(["python", "анализ%20данных"]) # Вызываем функцию habr с ключевыми словами
result

,date,title,link
0,7 часов назад,Функциональное программирование в Python: ежед...,/ru/companies/kaspersky/articles/762788/
1,29 сен в 15:16,Многопоточность в Python: очевидное и невероятное,/ru/articles/764420/
2,28 сен в 15:44,Построение пайплайна обработки данных в реальн...,/ru/companies/otus/articles/764136/
3,29 сен в 12:02,Как настроить сбор статистики и автоматическое...,/ru/companies/ispmanager/articles/764290/
4,3 окт в 19:17,FIFO очередь asyncio в Python,/ru/articles/764932/
5,3 окт в 03:21,Год ожиданий — и мы получили Python 3.12. Изме...,/ru/companies/selectel/articles/761914/
6,11 сен в 12:49,Геоаналитика на Python,/ru/companies/otus/articles/760148/
7,19 сен в 07:42,Microsoft представила дополнение Python Editor...,/ru/news/761862/
8,22 авг в 20:14,Microsoft добавила Python в Excel,/ru/news/756266/
9,вчера в 12:13,Пишем АПИ автотесты на Python по шагам,/ru/articles/765512/


#Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:



*   кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
*   в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:

  **<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>**

In [7]:
def habr(query, num_pages=1):
    df = pd.DataFrame()
    URL = 'https://habr.com/ru/search/'
    for q in query:
        for page in range(1, num_pages + 1):  # Проходим по каждой странице поисковой выдачи (от 1 до num_pages включительно)
            params = {
                'q': q,
                'page': page
            }
            req = requests.get(URL, params=params)
            time.sleep(0.3)
            soup = BeautifulSoup(req.text, 'html.parser')
            articles = soup.find_all('article', class_='tm-articles-list__item')
            for article in articles:
                title = article.find('h2', class_="tm-title tm-title_h2").text
                link = article.find('a', class_="tm-title__link").get('href')
                date = article.find('span', class_="tm-article-datetime-published").text
                row = {'date': date, 'title': title, 'link': link}
                df = pd.concat([df, pd.DataFrame([row])])

    return df.reset_index(drop=True)

result = habr(["python", "анализ%20данных"], num_pages=4) # Теперь указывая номер страницы, результат новой страницы прибавляеться к предыдущему df
result


,date,title,link
0,7 часов назад,Функциональное программирование в Python: ежед...,/ru/companies/kaspersky/articles/762788/
1,29 сен в 15:16,Многопоточность в Python: очевидное и невероятное,/ru/articles/764420/
2,28 сен в 15:44,Построение пайплайна обработки данных в реальн...,/ru/companies/otus/articles/764136/
3,29 сен в 12:02,Как настроить сбор статистики и автоматическое...,/ru/companies/ispmanager/articles/764290/
4,3 окт в 19:17,FIFO очередь asyncio в Python,/ru/articles/764932/
...,...,...,...
75,4 дек 2020 в 21:03,Python как компилируемый статически типизирова...,/ru/news/531402/
76,3 мар 2020 в 13:22,В начале этого года Python сместил Java и стал...,/ru/companies/itsumma/news/490834/
77,3 июн 2020 в 09:00,Вышла версия 0.0.2 snakeware — дистрибутива Li...,/ru/news/505096/
78,13 ноя 2020 в 10:10,Создатель Python Гвидо ван Россум выходит на р...,/ru/news/527858/


In [8]:
def habr(query, num_pages=1):
    df = pd.DataFrame()
    URL = 'https://habr.com/ru/search/'
    for q in query:
        for page in range(1, num_pages + 1): # Проходим по каждой странице поисковой выдачи (от 1 до num_pages включительно)
            params = {
                'q': q,
                'page': page
            }
            req = requests.get(URL, params=params)
            time.sleep(0.3)
            soup = BeautifulSoup(req.text, 'html.parser')
            articles = soup.find_all('article', class_='tm-articles-list__item')
            for article in articles:
                title = article.find('h2', class_="tm-title tm-title_h2").text
                link = article.find('a', class_="tm-title__link").get('href')
                date = article.find('span', class_="tm-article-datetime-published").text
                row = {'date': date, 'title': title, 'link': link}

                # Добавляем код для извлечения полного текста статьи
                full_text_req = requests.get(urllib.parse.urljoin(URL, link)).text
                full_text_soup = BeautifulSoup(full_text_req, 'html.parser')
                full_text = full_text_soup.find('div', class_='tm-article-body').text.strip()
                row['text'] = full_text

                # Добавляем код для извлечения количества лайков
                likes = article.find('div', class_='tm-votes-meter tm-data-icons__item').text
                row['likes'] = likes

                df = pd.concat([df, pd.DataFrame([row])])

    return df.reset_index(drop=True)

result = habr(["python", "анализ%20данных"], num_pages=1) # Теперь указывая номер страницы, результат новой страницы прибавляеться к предыдущему df
result


,date,title,link,text,likes
0,7 часов назад,Функциональное программирование в Python: ежед...,/ru/companies/kaspersky/articles/762788/,"Как говорится, спроси пять программистов, что ...",Всего голосов 6: ↑6 и ↓0 +6
1,29 сен в 15:16,Многопоточность в Python: очевидное и невероятное,/ru/articles/764420/,В данной статье я покажу на практическом приме...,Всего голосов 38: ↑36 и ↓2 +34
2,28 сен в 15:44,Построение пайплайна обработки данных в реальн...,/ru/companies/otus/articles/764136/,"Привет, Хабр!Обработка данных в реальном време...",Всего голосов 18: ↑16 и ↓2 +14
3,29 сен в 12:02,Как настроить сбор статистики и автоматическое...,/ru/companies/ispmanager/articles/764290/,"Меня зовут Вячеслав, и я руководитель отдела м...",Всего голосов 5: ↑5 и ↓0 +5
4,3 окт в 19:17,FIFO очередь asyncio в Python,/ru/articles/764932/,ПримечаниеЭто перевод статьи об asyncio.Queue....,Всего голосов 5: ↑5 и ↓0 +5
5,3 окт в 03:21,Год ожиданий — и мы получили Python 3.12. Изме...,/ru/companies/selectel/articles/761914/,Новую версию Python многие разработчики ожидал...,Всего голосов 83: ↑82 и ↓1 +81
6,11 сен в 12:49,Геоаналитика на Python,/ru/companies/otus/articles/760148/,Геоаналитика и пространственный анализ — это м...,Всего голосов 6: ↑4 и ↓2 +2
7,19 сен в 07:42,Microsoft представила дополнение Python Editor...,/ru/news/761862/,Microsoft представила дополнение Python Editor...,Всего голосов 3: ↑3 и ↓0 +3
8,22 авг в 20:14,Microsoft добавила Python в Excel,/ru/news/756266/,"22 августа 2023 года Microsoft сообщила, что д...",Всего голосов 24: ↑23 и ↓1 +22
9,вчера в 12:13,Пишем АПИ автотесты на Python по шагам,/ru/articles/765512/,Меня зовут Алексей. Я являюсь специалистом по ...,Всего голосов 1: ↑1 и ↓0 +1
